In [4]:
len((['Huechuraba' 'Lo Espejo' 'Quilin' 'Sucursal Miraflores' 'Coquimbo'
 'Hijuelas' 'Ovalle' 'San Antonio' 'San Felipe' 'Viña del Mar' 'Chillan'
 'Concepcion' 'Curico' 'Los Angeles' 'Rancagua' 'Talca' 'Castro'
 'Coyhaique' 'Osorno' 'Puerto Montt' 'Punta Arenas' 'Temuco' 'Valdivia'
 'Antofagasta' 'Arica' 'Calama' 'Copiapo' 'Iquique' 'Vallenar'
 'VD FoodService' 'VD Industrial' 'VD Grandes Clientes'
 'Sucursal Dieciocho' 'Vendedores San Pablo'][0]).split(' '))

17

In [8]:
len(['1020253' ,'1020042' ,'1020027' ,'1020013', '1020010', '1020003', '1020001'
 '1010695' ,'1010267' ,'1010078' ,'1010077', '1010048','1010002'])

12

In [10]:
12*17*5/60

17.0

# Información previa

## Diseño y esquema codigo

Predicción de prophet de la demanda y el precio

El group by debe estar en el codigo, para segmentarlo

1. Predecir **Demanda** al corto plazo

2. Predecir el **Precio** promedio ponderado `Ventas CLP.sum()/Demanda KG.sum()`

* Predecir al corto plazo, generando un resultado del error al predecir el año 2021, semana a semana.




## Apuntes Cod 

```{python}
Data['Peso neto KG']=Data['Peso neto'].str.split('KG', expand=True)[[0]]
Data.pop('Peso neto')
Data['Cantidad Unidades CJ']=Data['Cantidad Unidades'].str.split('CJ', expand=True)[[0]]
Data.pop('Cantidad Unidades')
Data['Precio Unitario CLP / KG']=Data['Precio Unitario'].str.split('CLP / KG', expand=True)[[0]]



    if(x[len(x)-3:]=='USD'):
        x=x.split(' USD')[1]
    elif(x[len(x)-3:]=='CLP'):
        x=x.split(' CLP')[1]


```

## Este es un extracto del paper:
 

	
Ventas pasadas con retraso  k, es decir, ventas diarias para el SKU considerado de los últimos k días (tw k : ventana de tiempo de tamaño k ).

	
* Variables binarias para caracterizar días especiales, como:
    * Pago , que caracteriza los días al final de cada mes cuando las personas reciben el pago mensual.
    * Pago intermedio , que caracteriza los días al final de cada quincena en que las personas reciben dos pagos semanales.
    * Antes de las vacaciones , caracterizando los días antes de las vacaciones.
    * Vacaciones , que caracterizan las vacaciones.
    * Independencia , que caracteriza los días de la independencia de Chile (18 y 19 de septiembre).
    * Santa , caracterizando los días de la semana antes del este.
    * Vacaciones , caracterizando los días que pertenecen al período de vacaciones de verano (enero y febrero).
    * Verano , que caracteriza los días de verano (1 de octubre-31 de marzo).	
    * Año nuevo (1 de enero), que caracteriza el único día de cierre de supermercados en Chile.
	
	
* Variables de precio:
    * Precio A  =  precio original del SKU considerado en Economax.
    * Precio B  =  (Precio A) / (Precio máximo en micro-mercado).
    * Precio C  =  (Precio A) / (Precio mínimo en micro-mercado).

		
Observación : Las variables “Precio máximo en micro-mercado” y “Precio mínimo en micro-mercado” son el precio máximo (mínimo) para un determinado SKU en el micro-mercado relevante y son proporcionados por una empresa de investigación de mercado.
		
	

	
 

# Package Install & Open Data


## Import

### import general 

In [1]:
# =============================================================================
# Importar paquetes y abrir Data
# =============================================================================

#gestion DF
import pandas as pd

#Funciones mat y matricicales
import numpy as np

#ploteo simple
import matplotlib.pyplot as plt

# Graficas bonitas
from plotnine import *

#para ajustar las fechas
import datetime

#para calcular el skew de un array() o list()
from scipy.stats import skew 

#mode function
import statistics 

from mizani.breaks import date_breaks
from mizani.formatters import date_format

### import and function 

In [2]:

import warnings
import pandas as pd
import matplotlib.pyplot as plt
# prophet model 
from fbprophet import Prophet
# prophet preformance
from fbprophet.diagnostics import cross_validation
from fbprophet.diagnostics import performance_metrics
from fbprophet.plot import plot_cross_validation_metric

# don't do this 
warnings.filterwarnings('ignore')
# "high resolution"
%config InlineBackend.figure_format = 'retina'

Importing plotly failed. Interactive plots will not work.


##  Data Facturación CT(New)
Esta data contiene las facturaciones(Ventas a los clientes del canal tradicional)

### Selección de la zona de estudio 

In [3]:
#Nombre del producto
Fecha='07_06_21'

# =============================================================================
# Ajuste de los datos
# =============================================================================
File_name='Fact_CL_Mater_Canales_'+Fecha
col_names=pd.read_csv("Data/"+File_name+".csv",
                    encoding="utf-8",sep=";",nrows=0).columns
types_dict = {col: str for col in col_names}

Data = pd.read_csv("Data/"+File_name+".csv",
                   encoding="utf-8",sep=";",dtype=types_dict)
Data.head(1)

,Material,Tipo de cliente,Zona de ventas,Zona de ventas.1,Oficina de Ventas Unifica,Oficina de Ventas Unifica.1,Mes,Año natural/Semana,Día natural,Día de la semana,Kilos Venta KG,Venta Neta,Precio Promedio
0,1020253,Tradicional,Z001,Santiago,Huechuraba,T101,1,01.2019,02.01.2019,MI,36,81.788 CLP,2.250 CLP


### Ajuste facturaciones

In [4]:
# =============================================================================
# Ajustar la data
# =============================================================================
def Ajuste_data(data):
    df=data.copy()
    # Funciones para eliminar los putos y cambiar las , por .
    def categ_uniq(x):
        return x[len(x)-3:]

    df['Unidad Venta']=df['Venta Neta'].apply(categ_uniq)
    df['Unidad Precio']=df['Precio Promedio'].apply(categ_uniq)
    
    def CLP_Dolar(x):
        x=str(x)
        x=x.replace('.', '')
        x=x.replace(',', '.')
        if(x[len(x)-3:]=='USD'):
            x=x.split(' USD')[0]
            x=float(x)
            #x=x*720
        elif(x[len(x)-3:]=='CLP'):
            x=x.split(' CLP')[0]
            x=float(x)
        return x
    
    def change_num_point(x):
        x=str(x)
        return x.replace('.', '')
    def change_num_coma(x):
        x=str(x)
        return x.replace(',', '.')
    def delete_asterisco(x):
        x=str(x)
        return x.replace('*','0' )
    
    #proceso iterativo, 
    for i in ['Kilos Venta KG','Venta Neta','Precio Promedio']:
    # se elimina el punto definido para los miles
        if(i!='Kilos Venta KG'):
            df[i]=df[i].apply(CLP_Dolar)
        else:
            df[i]=df[i].apply(change_num_point)
            # se reemplaza la coma por un punto, para definir los decimales correctamente
            df[i]=df[i].apply(change_num_coma)
        # se elimina el valor * 
        df[i]=df[i].apply(delete_asterisco)
        df[i]=df[i].astype('float64')
        # Se pasa de ser un str a un int
        print(i)
        #df[i]=df[i].astype('int64')

    #Separacion fecha
    df['Semana']=df['Año natural/Semana'].str.split('.', expand=True)[[0]]
    df['Año']=df['Año natural/Semana'].str.split('.', expand=True)[[1]]

    df['Semana']=df['Semana'].astype('int64')
    df['Año']=df['Año'].astype('int64')
    
    #
    df["Día natural"] = df["Día natural"].apply(lambda x: 
                                                datetime.datetime.strptime(x,
                                                                           '%d.%m.%Y').date())
    df['Año']=df['Año'].astype('int64')
    return df

In [5]:
data=Ajuste_data(Data)
data.head(3)
#data=<data[data['Año']>2018]

Kilos Venta KG
Venta Neta
Precio Promedio


,Material,Tipo de cliente,Zona de ventas,Zona de ventas.1,Oficina de Ventas Unifica,Oficina de Ventas Unifica.1,Mes,Año natural/Semana,Día natural,Día de la semana,Kilos Venta KG,Venta Neta,Precio Promedio,Unidad Venta,Unidad Precio,Semana,Año
0,1020253,Tradicional,Z001,Santiago,Huechuraba,T101,1,01.2019,2019-01-02,MI,36.0,81788.0,2250.0,CLP,CLP,1,2019
1,1020253,Tradicional,Z001,Santiago,Huechuraba,T101,1,01.2019,2019-01-03,JU,333.0,583204.0,1750.0,CLP,CLP,1,2019
2,1020253,Tradicional,Z001,Santiago,Huechuraba,T101,1,01.2019,2019-01-05,SA,219.0,397031.0,1815.0,CLP,CLP,1,2019


# Predicción  Prophet


In [6]:
y_col='Kilos Venta KG'
y_value='Volumen de ventas en KG'
y_title= 'Demanda'+'del canal tradicion'

## Descripción 


1. Predicción de Demanda dia
    
   * Evaluación a largo plazo
        1. Predicción año 2021 completo
    
   * Evaluación a corto plazo
        2. iteración mes a mes
        3. iteración semana a semana

2. Predicción del Precio Replica del de Demanda


Dataframe a considerar
```python

# posee las ventas de la zona de temuco del canal tradicional, debido al ajuste de zona
data


#DF_ventas Posee las ventas de los clientes del CT
DF_ventas 

#DF_VS Posee los niveles de Stok Abastecimiento y Demanda de la Z001
DF_VS

# Dias_Festivos Posee los dias festivos del 2019 al 2020
Dias_Festivos
```

Se juntaran los 3 df para formar 1, para ello se generará en primer lugar un df con

```python
DF_ventas[DF_ventas['identified_clusters']==4].groupby('Día natural').sum().reset_index()[['Día natural','Kilos Venta KG']]
``` 


De tal forma que se tiene la demanda dia del segmento, Despues se realiza un Merge con cada uno de los otros DF, de tal forma que la demanda dia posee más partes X que le agregan robustes al modelo.


## Construcción de la Data

In [7]:
data.head()

,Material,Tipo de cliente,Zona de ventas,Zona de ventas.1,Oficina de Ventas Unifica,Oficina de Ventas Unifica.1,Mes,Año natural/Semana,Día natural,Día de la semana,Kilos Venta KG,Venta Neta,Precio Promedio,Unidad Venta,Unidad Precio,Semana,Año
0,1020253,Tradicional,Z001,Santiago,Huechuraba,T101,1,01.2019,2019-01-02,MI,36.0,81788.0,2250.0,CLP,CLP,1,2019
1,1020253,Tradicional,Z001,Santiago,Huechuraba,T101,1,01.2019,2019-01-03,JU,333.0,583204.0,1750.0,CLP,CLP,1,2019
2,1020253,Tradicional,Z001,Santiago,Huechuraba,T101,1,01.2019,2019-01-05,SA,219.0,397031.0,1815.0,CLP,CLP,1,2019
3,1020253,Tradicional,Z001,Santiago,Huechuraba,T101,1,02.2019,2019-01-07,LU,128.0,258591.0,2017.0,CLP,CLP,2,2019
4,1020253,Tradicional,Z001,Santiago,Huechuraba,T101,1,02.2019,2019-01-08,MA,95.0,210790.0,2223.0,CLP,CLP,2,2019


In [8]:
data.columns

Index(['Material', 'Tipo de cliente', 'Zona de ventas', 'Zona de ventas.1',
       'Oficina de Ventas Unifica', 'Oficina de Ventas Unifica.1', 'Mes',
       'Año natural/Semana', 'Día natural', 'Día de la semana',
       'Kilos Venta KG', 'Venta Neta', 'Precio Promedio', 'Unidad Venta',
       'Unidad Precio', 'Semana', 'Año'],
      dtype='object')

In [9]:
print(data['Tipo de cliente'].unique())
print(data['Zona de ventas.1'].unique())
print(data['Oficina de Ventas Unifica'].unique())
print(data['Material'].unique())

['Tradicional' 'Industriales' 'Grandes Clientes' 'Foodservice']
['Santiago' 'Centro Norte' 'Centro Sur' 'Sur' 'Norte' 'Venta Directa']
['Huechuraba' 'Lo Espejo' 'Quilin' 'Sucursal Miraflores' 'Coquimbo'
 'Hijuelas' 'Ovalle' 'San Antonio' 'San Felipe' 'Viña del Mar' 'Chillan'
 'Concepcion' 'Curico' 'Los Angeles' 'Rancagua' 'Talca' 'Castro'
 'Coyhaique' 'Osorno' 'Puerto Montt' 'Punta Arenas' 'Temuco' 'Valdivia'
 'Antofagasta' 'Arica' 'Calama' 'Copiapo' 'Iquique' 'Vallenar'
 'VD FoodService' 'VD Industrial' 'VD Grandes Clientes'
 'Sucursal Dieciocho' 'Vendedores San Pablo']
['1020253' '1020042' '1020027' '1020013' '1020010' '1020003' '1020001'
 '1010695' '1010267' '1010078' '1010077' '1010048' '1010002']


## Iter Por Zona 

### función 

In [10]:
def Df_ubic(data_X,col,Str):
    df=data_X.copy()
    df=df[df[col]==Str]
    return df

In [11]:
def Predict_sem(D_Train,periodos=5):
    D_Train.columns = ['ds','y']
    
    # convert dates from string to datetime
    D_Train.ds = pd.to_datetime(D_Train.ds)
    #print(D_Train.dtypes)
    
    # set prophet model 
    prophet = Prophet(changepoint_prior_scale=0.15, yearly_seasonality=True)
    
    # fit $wmt data to model
    prophet.fit(D_Train)
    
    
    
    # build future dataframe for 5 years
    build_forecast = prophet.make_future_dataframe(periods= periodos , freq='D')
    #prophet.make_future_dataframe(periods= periodos, freq='D')
    
    # forecast future df w/ model
    forecast = prophet.predict(build_forecast)
    
    return(forecast)

### iteración 

In [17]:
df2 = pd.DataFrame(columns=['ds', 'yhat_lower', 'yhat_upper', 'yhat', 'Material','Oficina de Ventas Unifica', 'Zona de ventas.1'])
df2

,ds,yhat_lower,yhat_upper,yhat,Material,Oficina de Ventas Unifica,Zona de ventas.1


In [18]:
Listado_de_zonas=data['Zona de ventas.1'].unique
#for z in Listado_de_zonas:
Z='Norte'

dfz=Df_ubic(data,'Zona de ventas.1',Z)

Listado_de_Sucursales=dfz['Oficina de Ventas Unifica'].unique()

#for y in Listado_de_Sucursales:

y='Arica'
dfy=Df_ubic(dfz,'Oficina de Ventas Unifica',y)


Listado_de_Materiales=dfy['Material'].unique()

#for m in Listado_de_Materiales:
m=Listado_de_Materiales[0]
dfm=Df_ubic(dfy,'Material',m).groupby('Día natural').sum().reset_index()[['Día natural',y_col]]

dfmPred=Predict_sem(dfm,7)[['ds', 'yhat_lower', 'yhat_upper', 'yhat']].tail(6)
dfmPred['Material']=[m for i in range(dfmPred.shape[0])]
dfmPred['Oficina de Ventas Unifica']=[y for i in range(dfmPred.shape[0])]
dfmPred['Zona de ventas.1']=[Z for i in range(dfmPred.shape[0])]
dfmPred=dfmPred.reset_index()
dfmPred.pop('index')
df2=pd.concat([df2, dfmPred])
df2.head()

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


,ds,yhat_lower,yhat_upper,yhat,Material,Oficina de Ventas Unifica,Zona de ventas.1
0,2021-06-07,-62.972060,137.409989,39.029265,1020253,Arica,Norte
1,2021-06-08,-78.798542,124.459982,26.334752,1020253,Arica,Norte
2,2021-06-09,-41.022710,165.599132,60.806764,1020253,Arica,Norte
3,2021-06-10,-56.266824,145.219321,40.848031,1020253,Arica,Norte
4,2021-06-11,-56.122241,143.242394,44.408593,1020253,Arica,Norte


# Ejecución 

## Iteración 1 zona

In [19]:
df2 = pd.DataFrame(columns=['ds', 'yhat_lower', 'yhat_upper', 'yhat', 'Material','Oficina de Ventas Unifica', 'Zona de ventas.1'])
df2

,ds,yhat_lower,yhat_upper,yhat,Material,Oficina de Ventas Unifica,Zona de ventas.1


In [20]:
from time import time

In [23]:
df2 = pd.DataFrame(columns=['ds', 'yhat_lower', 'yhat_upper', 'yhat', 'Material','Oficina de Ventas Unifica', 'Zona de ventas.1'])

Listado_de_zonas=data['Zona de ventas.1'].unique

start_time = time()



#for Z in Listado_de_zonas:
Z='Norte'

dfz=Df_ubic(data,'Zona de ventas.1',Z)

Listado_de_Sucursales=dfz[dfz['Día natural'] >datetime.date(2021, 6, 2)]['Oficina de Ventas Unifica'].unique()

for y in Listado_de_Sucursales:
    #y='Huechuraba'
    dfy=Df_ubic(dfz,'Oficina de Ventas Unifica',y)


    Listado_de_Materiales=dfy['Material'].unique()

    for m in Listado_de_Materiales:
        #m=Listado_de_Materiales[0]
        material_time=time()
        print(y,m)


        dfm=Df_ubic(dfy,'Material',m).groupby('Día natural').sum().reset_index()[['Día natural',y_col]]

        dfmPred=Predict_sem(dfm,7)[['ds', 'yhat_lower', 'yhat_upper', 'yhat']].tail(6)
        dfmPred['Material']=[m for i in range(dfmPred.shape[0])]
        dfmPred['Oficina de Ventas Unifica']=[y for i in range(dfmPred.shape[0])]
        dfmPred['Zona de ventas.1']=[Z for i in range(dfmPred.shape[0])]
        dfmPred=dfmPred.reset_index()
        dfmPred.pop('index')
        df2=pd.concat([df2, dfmPred])
        
        print('El producto demoró',(time() - material_time),Z,y)
        print('tiempo que lleva corriendo',(time() - start_time)/60,'Min')
    
print('tiempo bloque completo',(time() - start_time)/60,'Min')
df2.head()

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Antofagasta 1020253


INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 0.


El producto demoró 3.0505330562591553 Norte Antofagasta
tiempo que lleva corriendo 0.055392336845397946 Min
Antofagasta 1020042


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


...

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


El producto demoró 4.728689432144165 Norte Copiapo
tiempo que lleva corriendo 3.911816084384918 Min
Copiapo 1010048


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


El producto demoró 4.433742046356201 Norte Copiapo
tiempo que lleva corriendo 3.9857201099395754 Min
Copiapo 1010002
El producto demoró 5.248722314834595 Norte Copiapo
tiempo que lleva corriendo 4.073223765691122 Min
tiempo bloque completo 4.073223765691122 Min


,ds,yhat_lower,yhat_upper,yhat,Material,Oficina de Ventas Unifica,Zona de ventas.1
0,2021-06-06,-213.009809,324.855873,47.275884,1020253,Antofagasta,Norte
1,2021-06-07,-199.780639,331.531749,76.464918,1020253,Antofagasta,Norte
2,2021-06-08,-175.008560,353.134381,74.623993,1020253,Antofagasta,Norte
3,2021-06-09,-148.680240,375.212586,103.725185,1020253,Antofagasta,Norte
4,2021-06-10,-114.364882,435.328510,153.321113,1020253,Antofagasta,Norte


In [43]:
165.4798605442047/60

2.7579976757367453

### CSV SAVE 

In [55]:
max_time=df2.ds.max()
max_time=str(max_time).split(' ')[0]
max_time

'2021-06-12'

In [59]:
#Error_DF_Sum.to_csv('E_interv_Dem.csv', index=False)
df2.to_csv('Predicción Zona '+Z+max_time+'.csv', index=False)

## Iteración completa

In [13]:
from time import time

In [25]:
df2 = pd.DataFrame(columns=['ds', 'yhat_lower', 'yhat_upper', 'yhat', 'Material','Oficina de Ventas Unifica', 'Zona de ventas.1'])

Listado_de_zonas=data['Zona de ventas.1'].unique()

start_time = time()



for Z in Listado_de_zonas:
#Z='Santiago'
    print(Z)
    dfz=Df_ubic(data,'Zona de ventas.1',Z)

    Listado_de_Sucursales=dfz[dfz['Día natural'] 
                              >datetime.date(2021, 6, 2)][
        'Oficina de Ventas Unifica'].unique()

    for y in Listado_de_Sucursales:
        #y='Huechuraba'
        print(y)
        dfy=Df_ubic(dfz,'Oficina de Ventas Unifica',y)


        Listado_de_Materiales=dfy[dfy.Año==2021].Material.unique()

        for m in Listado_de_Materiales:
            #m=Listado_de_Materiales[0]
            material_time=time()
            #print(y,m)


            dfm=Df_ubic(dfy,
                        'Material',m).groupby('Día natural'
                                                 ).sum().reset_index()[['Día natural',y_col]]

            dfmPred=Predict_sem(dfm,7)[['ds', 'yhat_lower',
                                        'yhat_upper', 'yhat']].tail(6)
            
            dfmPred['Material']=[m for i in range(dfmPred.shape[0])]
            dfmPred['Oficina de Ventas Unifica']=[y for i in range(dfmPred.shape[0])]
            dfmPred['Zona de ventas.1']=[Z for i in range(dfmPred.shape[0])]
            dfmPred=dfmPred.reset_index()
            dfmPred.pop('index')
            df2=pd.concat([df2, dfmPred])
            print('El producto demoró',(time() - material_time),Z,y)
            print('tiempo que lleva corriendo',(time() - start_time)/60, 'Min')
    
print('tiempo bloque completo',(time() - start_time)/60, 'Min')


max_time=df2.ds.max()
max_time=str(max_time).split(' ')[0]


df2.to_csv('Predicción CL ' +max_time+ '.csv', index=False)
df2.head()


Santiago


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Huechuraba


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


El producto demoró 4.026480436325073 Santiago Huechuraba
tiempo que lleva corriendo 0.07164204915364583 Min


...

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


El producto demoró 2.823758125305176 Venta Directa VD FoodService
tiempo que lleva corriendo 20.293634418646494 Min


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


El producto demoró 3.557161569595337 Venta Directa VD FoodService
tiempo que lleva corriendo 20.352928471565246 Min
El producto demoró 2.677013635635376 Venta Directa VD FoodService
tiempo que lleva corriendo 20.397562003135683 Min
tiempo bloque completo 20.397562003135683 Min


,ds,yhat_lower,yhat_upper,yhat,Material,Oficina de Ventas Unifica,Zona de ventas.1
0,2021-06-07,-203.048531,1318.056941,545.820125,1020253,Huechuraba,Santiago
1,2021-06-08,-38.219171,1487.247057,705.888159,1020253,Huechuraba,Santiago
2,2021-06-09,28.889193,1644.230881,832.462643,1020253,Huechuraba,Santiago
3,2021-06-10,192.190616,1837.272069,984.431776,1020253,Huechuraba,Santiago
4,2021-06-11,276.264925,1796.591032,1028.985547,1020253,Huechuraba,Santiago


### Exploracion del error 

In [28]:
Z

'Venta Directa'

In [27]:
y

'VD FoodService'

In [26]:
m

'1010048'

In [79]:
dfy['Material'].unique()

array(['1020253', '1020027', '1020013', '1020010', '1020003', '1020001',
       '1010695', '1010267', '1010078', '1010077', '1010048', '1010002'],
      dtype=object)

In [78]:
dfy[dfy.Año==2021]['Material'].unique()

array(['1020253', '1020027', '1020013', '1020010', '1020003', '1020001',
       '1010695', '1010267', '1010078', '1010077', '1010048', '1010002'],
      dtype=object)

In [81]:
dfy[dfy.Año==2021].groupby('Material').count()

,Tipo de cliente,Zona de ventas,Zona de ventas.1,Oficina de Ventas Unifica,Oficina de Ventas Unifica.1,Mes,Año natural/Semana,Día natural,Día de la semana,Kilos Venta KG,Venta Neta,Precio Promedio,Unidad Venta,Unidad Precio,Semana,Año
Material,,,,,,,,,,,,,,,,
1010002,211,211,211,211,211,211,211,211,211,211,211,211,211,211,211,211
1010048,273,273,273,273,273,273,273,273,273,273,273,273,273,273,273,273
1010077,104,104,104,104,104,104,104,104,104,104,104,104,104,104,104,104
1010078,314,314,314,314,314,314,314,314,314,314,314,314,314,314,314,314
1010267,310,310,310,310,310,310,310,310,310,310,310,310,310,310,310,310
1010695,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14
1020001,117,117,117,117,117,117,117,117,117,117,117,117,117,117,117,117
1020003,68,68,68,68,68,68,68,68,68,68,68,68,68,68,68,68
1020010,197,197,197,197,197,197,197,197,197,197,197,197,197,197,197,197


In [72]:
dfy.groupby('Material').count()

,Tipo de cliente,Zona de ventas,Zona de ventas.1,Oficina de Ventas Unifica,Oficina de Ventas Unifica.1,Mes,Año natural/Semana,Día natural,Día de la semana,Kilos Venta KG,Venta Neta,Precio Promedio,Unidad Venta,Unidad Precio,Semana,Año
Material,,,,,,,,,,,,,,,,
1010002,1031,1031,1031,1031,1031,1031,1031,1031,1031,1031,1031,1031,1031,1031,1031,1031
1010048,1617,1617,1617,1617,1617,1617,1617,1617,1617,1617,1617,1617,1617,1617,1617,1617
1010077,648,648,648,648,648,648,648,648,648,648,648,648,648,648,648,648
1010078,1695,1695,1695,1695,1695,1695,1695,1695,1695,1695,1695,1695,1695,1695,1695,1695
1010267,1475,1475,1475,1475,1475,1475,1475,1475,1475,1475,1475,1475,1475,1475,1475,1475
1010695,359,359,359,359,359,359,359,359,359,359,359,359,359,359,359,359
1020001,707,707,707,707,707,707,707,707,707,707,707,707,707,707,707,707
1020003,450,450,450,450,450,450,450,450,450,450,450,450,450,450,450,450
1020010,984,984,984,984,984,984,984,984,984,984,984,984,984,984,984,984


In [83]:
dfy_21=dfy[dfy.Año==2021]

In [85]:
dfy_21[dfy_21.Material=='1010695']

,Material,Tipo de cliente,Zona de ventas,Zona de ventas.1,Oficina de Ventas Unifica,Oficina de Ventas Unifica.1,Mes,Año natural/Semana,Día natural,Día de la semana,Kilos Venta KG,Venta Neta,Precio Promedio,Unidad Venta,Unidad Precio,Semana,Año
220172,1010695,Tradicional,Z005,Norte,Arica,T161,1,02.2021,2021-01-14,JU,21.0,21260.0,1000.0,CLP,CLP,2,2021
220212,1010695,Tradicional,Z005,Norte,Arica,T161,3,11.2021,2021-03-20,SA,22.0,23865.0,1110.0,CLP,CLP,11,2021
220235,1010695,Tradicional,Z005,Norte,Arica,T161,4,15.2021,2021-04-13,MA,107.0,97474.0,912.0,CLP,CLP,15,2021
220251,1010695,Tradicional,Z005,Norte,Arica,T161,5,18.2021,2021-05-07,VI,108.0,102772.0,950.0,CLP,CLP,18,2021
220252,1010695,Tradicional,Z005,Norte,Arica,T161,5,18.2021,2021-05-08,SA,65.0,61674.0,950.0,CLP,CLP,18,2021
220253,1010695,Tradicional,Z005,Norte,Arica,T161,5,19.2021,2021-05-10,LU,42.0,40280.0,950.0,CLP,CLP,19,2021
220254,1010695,Tradicional,Z005,Norte,Arica,T161,5,19.2021,2021-05-11,MA,195.0,158591.0,815.0,CLP,CLP,19,2021
220255,1010695,Tradicional,Z005,Norte,Arica,T161,5,19.2021,2021-05-12,MI,22.0,23294.0,1055.0,CLP,CLP,19,2021
220256,1010695,Tradicional,Z005,Norte,Arica,T161,5,20.2021,2021-05-20,JU,106.0,102184.0,960.0,CLP,CLP,20,2021
220257,1010695,Tradicional,Z005,Norte,Arica,T161,5,21.2021,2021-05-26,MI,109.0,124844.0,1150.0,CLP,CLP,21,2021


In [29]:
429.91*692.60

297755.666

### CSV SAVE 

In [68]:
df2.to_csv('Predicción CL ' +max_time+ '.csv', index=False)
df2.tail()

,ds,yhat_lower,yhat_upper,yhat,Material,Oficina de Ventas Unifica,Zona de ventas.1
1,2021-06-08,-69.517433,129.412460,26.334752,1020253,Arica,Norte
2,2021-06-09,-37.105720,167.423934,60.806764,1020253,Arica,Norte
3,2021-06-10,-57.877456,133.962036,40.848031,1020253,Arica,Norte
4,2021-06-11,-54.582450,142.198912,44.408593,1020253,Arica,Norte
5,2021-06-12,-46.974651,154.307306,59.735489,1020253,Arica,Norte


In [55]:
max_time=df2.ds.max()
max_time=str(max_time).split(' ')[0]
max_time

'2021-06-12'

In [59]:
#Error_DF_Sum.to_csv('E_interv_Dem.csv', index=False)
df2.to_csv('Predicción Zona '+Z+max_time+'.csv', index=False)

# Iteración Prophet Calculo del Error

se busca realizar una prediccion recuerrente de los dias del 2021
para ello se tiene que predecir con el dia 2019 ... 2021-1-1 el dia 2021-1-2 e ir aumentando la data hasta llegar al  2021-2-11

### Sin iteración predicción con conocimiento perfeccto

In [ ]:
df_P=DF_prophet[['Día natural', y_col]].copy()
df_P.columns = ['ds','y']

#FECHA DE INICIO
df_P=df_P[df_P.ds>=datetime.date(2019, 1, 1)]
df_P.head(1)
#print_simpl_graph(df_P)

###  Particion

In [ ]:
i=1
#FECHA DE TERMINO
df_P_train=df_P[df_P.ds<datetime.date(2021, i, 1)].copy()

#FECHA DE PRUEBA
df_P_test=df_P[df_P.ds>datetime.date(2021, i, 1)].copy()
df_P_test=df_P_test[df_P_test.ds<datetime.date(2021, i+1, 1)]
# convert dates from string to datetime
df_P_train.ds = pd.to_datetime(df_P_train.ds)
# convert dates from string to datetime
df_P_test.ds = pd.to_datetime(df_P_test.ds)
#print(df_P_test.dtypes,df_P_train.dtypes)

print('tail train:',df_P_train.tail(1).ds.iloc[0])
print('tail test:',df_P_test.tail(1).ds.iloc[0])

### funciónes

#### Predicción

In [ ]:
def Predict_Prophet(D_Train,D_Test,periodos):
    
    #D_propht:= Df utilizado para predecir
    # ds es date
    # y es la variable a predecir
    
    
    
    
    # column names for prophet compliance
    D_Train.columns = ['ds','y']
    #D_propht.head(3)
    
    # convert dates from string to datetime
    D_Train.ds = pd.to_datetime(D_Train.ds)
    print(D_Train.dtypes)
    
    """
    #imprime la grafica de dispersión de los datos
    print_simpl_graph(D_Train)
    print_simpl_graph(D_Test)
    """
    # set prophet model 
    prophet = Prophet(changepoint_prior_scale=0.15, yearly_seasonality=True)
    
    # fit $wmt data to model
    prophet.fit(D_Train)
    
    
    
    
    largo=D_Test.tail(1).ds.iloc[0]-D_Train.tail(1).ds.iloc[0]
    #prophet.make_future_dataframe(periods= abs(largo.days)+5 , freq='D').tail()
    print(D_Train.tail(1).ds.iloc[0])
    print(D_Test.tail(1).ds.iloc[0])
    print(largo)
    
    
    # build future dataframe for 5 years
    build_forecast = prophet.make_future_dataframe(periods= abs(largo.days) , freq='D')
    #prophet.make_future_dataframe(periods= periodos, freq='D')
    
    # forecast future df w/ model
    forecast = prophet.predict(build_forecast)
    
    
    
    """
    #plot forecasts
    prophet.plot(forecast, xlabel='Date', ylabel= y_value)
    plt.plot(D_Test.ds, D_Test.y, color='red', marker='o', linestyle=':',markersize=4,)
    plt.title('Comportamiento '+ y_title)
    # display graph
    plt.show()

    
    # tell us more about the forecast
    prophet.plot_components(forecast)
    """
    return(prophet,build_forecast,forecast)

#### Error

In [ ]:
def Error_Pred(DF_Predict,df_P_test):
    DF_E=pd.merge(df_P_test,DF_Predict
                     , how='left')
    DF_E['Error']=DF_E['yhat']-DF_E.y
    
    DF_E['Error %']=DF_E['yhat']/DF_E.y-1
    """
    DF_E.Error.hist()
    plt.show()
    """
    
    tempo=DF_E.copy()
    tempo['Semana']=tempo.ds.apply(lambda x: x.week).astype('category')
    tempo['Mes']=tempo.ds.apply(lambda x: x.month).astype('category')
    tempo.Error=tempo.Error.astype('float64')#.abs()
    print('Error esperado',tempo.Error.sum()/tempo.shape[0])
    
    tmp=tempo.groupby(['Mes','Semana']).sum().reset_index()
    # como es categorico, generá valores 0 para las columnas 
    #que no tienen registros en el mes.
    #para evitar esto, se filtran los valores menos a 0.
    
    
    tmp=tmp[tmp.y > 0]
    print(tmp.shape)
    
    """
    tmp.Error.hist()
    plt.show()
    """
    
    return(tmp)

### Predicciones siguientes 

In [ ]:
Resultados=Predict_Prophet(df_P_train,df_P_test,0)

In [ ]:
DF_Predict=Resultados[2][['ds','yhat']]#.reset_index()
DF_Predict.head()

In [ ]:
Error_DF_Sum=Error_Pred(DF_Predict,df_P_test)
Error_DF_Sum

#### iteracion demas meses 

In [ ]:
for i in range(2,6):
    print('\n',i)
    #FECHA DE TERMINO
    df_P_train=df_P[df_P.ds<datetime.date(2021, i, 1)].copy()

    #FECHA DE PRUEBA
    df_P_test=df_P[df_P.ds>datetime.date(2021, i, 1)].copy()
    df_P_test=df_P_test[df_P_test.ds<datetime.date(2021, i+1, 1)]
    # convert dates from string to datetime
    df_P_train.ds = pd.to_datetime(df_P_train.ds)
    # convert dates from string to datetime
    df_P_test.ds = pd.to_datetime(df_P_test.ds)
    #print(df_P_test.dtypes,df_P_train.dtypes)

    print('tail train:',df_P_train.tail(1).ds.iloc[0])
    print('tail test:',df_P_test.tail(1).ds.iloc[0])
    
    Resultados=Predict_Prophet(df_P_train,df_P_test,0)
    DF_Predict=Resultados[2][['ds','yhat']]#.reset_index()
    di=Error_Pred(DF_Predict,df_P_test)
    Error_DF_Sum=pd.concat([Error_DF_Sum, di])

In [ ]:
Error_DF_Sum.Mes=Error_DF_Sum.Mes.astype('category')

In [ ]:
ggplot(Error_DF_Sum, aes(x=Error_DF_Sum.Semana,y='Error'))+\
geom_col(aes(fill='Mes'))

In [ ]:
Error_DF_Sum['Error %']=Error_DF_Sum.yhat/Error_DF_Sum.y-1
Error_DF_Sum.head()

In [ ]:
ggplot(Error_DF_Sum[Error_DF_Sum['Error %']<=2], aes(x='Semana',y='Error %'))+\
geom_col(aes(fill='Mes'))

In [ ]:
Error_DF_Sum.Error.describe()

#### group mensual 

In [ ]:
tmp2=Error_DF_Sum.copy()
tmp2.Error=tmp2.Error.astype('float64')#.abs()
tmp2.Mes=tmp2.Mes.astype('category')

print(tmp2.Error.mean())

dx=tmp2.groupby('Mes').sum().reset_index()#['Mes','Error']
dx

In [ ]:
ggplot(dx, aes(x=dx.Mes,y='Error'))+\
geom_col(aes(fill='Mes'))

In [ ]:
ggplot(dx, aes(x=dx.Mes,y='Error %'))+\
geom_col(aes(fill='Mes'))

In [ ]:
dx.Error.hist()

## csv create

###  Particion

In [ ]:
i=1
#FECHA DE TERMINO
df_P_train=df_P[df_P.ds<datetime.date(2021, i, 1)].copy()

#FECHA DE PRUEBA
df_P_test=df_P[df_P.ds>datetime.date(2021, i, 1)].copy()
df_P_test=df_P_test[df_P_test.ds<datetime.date(2021, i+1, 1)]
# convert dates from string to datetime
df_P_train.ds = pd.to_datetime(df_P_train.ds)
# convert dates from string to datetime
df_P_test.ds = pd.to_datetime(df_P_test.ds)
#print(df_P_test.dtypes,df_P_train.dtypes)

print('tail train:',df_P_train.tail(1).ds.iloc[0])
print('tail test:',df_P_test.tail(1).ds.iloc[0])

### funciónes

#### Predicción

In [ ]:
def Predict_Prophet(D_Train,D_Test,periodos):
    
    #D_propht:= Df utilizado para predecir
    # ds es date
    # y es la variable a predecir
    
    
    
    
    # column names for prophet compliance
    D_Train.columns = ['ds','y']
    #D_propht.head(3)
    
    # convert dates from string to datetime
    D_Train.ds = pd.to_datetime(D_Train.ds)
    print(D_Train.dtypes)
    
    """
    #imprime la grafica de dispersión de los datos
    print_simpl_graph(D_Train)
    print_simpl_graph(D_Test)
    """
    # set prophet model 
    prophet = Prophet(changepoint_prior_scale=0.15, yearly_seasonality=True)
    
    # fit $wmt data to model
    prophet.fit(D_Train)
    
    
    
    
    largo=D_Test.tail(1).ds.iloc[0]-D_Train.tail(1).ds.iloc[0]
    #prophet.make_future_dataframe(periods= abs(largo.days)+5 , freq='D').tail()
    print(D_Train.tail(1).ds.iloc[0])
    print(D_Test.tail(1).ds.iloc[0])
    print(largo)
    
    
    # build future dataframe for 5 years
    build_forecast = prophet.make_future_dataframe(periods= abs(largo.days) , freq='D')
    #prophet.make_future_dataframe(periods= periodos, freq='D')
    
    # forecast future df w/ model
    forecast = prophet.predict(build_forecast)
    
    
    
    """
    #plot forecasts
    prophet.plot(forecast, xlabel='Date', ylabel= y_value)
    plt.plot(D_Test.ds, D_Test.y, color='red', marker='o', linestyle=':',markersize=4,)
    plt.title('Comportamiento '+ y_title)
    # display graph
    plt.show()

    
    # tell us more about the forecast
    prophet.plot_components(forecast)
    """
    return(prophet,build_forecast,forecast)

#### Error

In [ ]:
def Error_Pred(DF_Predict,df_P_test):
    DF_E=pd.merge(df_P_test,DF_Predict
                     , how='left')
    DF_E['Error']=DF_E['yhat']-DF_E.y
    
    DF_E['Error %']=DF_E['yhat']/DF_E.y-1
    """
    DF_E.Error.hist()
    plt.show()
    """
    
    tempo=DF_E.copy()
    tempo['Semana']=tempo.ds.apply(lambda x: x.week).astype('category')
    tempo['Mes']=tempo.ds.apply(lambda x: x.month).astype('category')
    tempo.Error=tempo.Error.astype('float64')#.abs()

    """
    tmp.Error.hist()
    plt.show()
    """
    
    return(tempo)

### Predicciones siguientes 

In [ ]:
Resultados=Predict_Prophet(df_P_train,df_P_test,0)

In [ ]:
DF_Predict=Resultados[2][['ds','yhat']]#.reset_index()
DF_Predict.head()

In [ ]:
Error_DF_Sum=Error_Pred(DF_Predict,df_P_test)
Error_DF_Sum

#### iteracion demas meses 

In [ ]:
for i in range(2,6):
    print('\n',i)
    #FECHA DE TERMINO
    df_P_train=df_P[df_P.ds<datetime.date(2021, i, 1)].copy()

    #FECHA DE PRUEBA
    df_P_test=df_P[df_P.ds>datetime.date(2021, i, 1)].copy()
    df_P_test=df_P_test[df_P_test.ds<datetime.date(2021, i+1, 1)]
    # convert dates from string to datetime
    df_P_train.ds = pd.to_datetime(df_P_train.ds)
    # convert dates from string to datetime
    df_P_test.ds = pd.to_datetime(df_P_test.ds)
    #print(df_P_test.dtypes,df_P_train.dtypes)

    print('tail train:',df_P_train.tail(1).ds.iloc[0])
    print('tail test:',df_P_test.tail(1).ds.iloc[0])
    
    Resultados=Predict_Prophet(df_P_train,df_P_test,0)
    DF_Predict=Resultados[2][['ds','yhat']]#.reset_index()
    di=Error_Pred(DF_Predict,df_P_test)
    Error_DF_Sum=pd.concat([Error_DF_Sum, di])

In [ ]:
Error_DF_Sum.Mes=Error_DF_Sum.Mes.astype('category')

In [ ]:
ggplot(Error_DF_Sum, aes(x=Error_DF_Sum.ds,y='Error'))+\
geom_col(aes(fill='Mes'))+\
theme(axis_text_x = element_text(angle=90, hjust=1))#,figure_size=(26,10)


In [ ]:
Error_DF_Sum.Error.describe()

In [ ]:
Error_DF_Sum.Error.hist()

In [ ]:
#Error_DF_Sum.to_csv('E_interv_Dem.csv', index=False)

In [ ]:
Error_DF_Sum['Semana']=Error_DF_Sum['Semana'].astype('category')
x=2
[(ggplot(Error_DF_Sum[Error_DF_Sum.Mes==str(x)], aes(x='ds',y='Error'))+\
geom_col(aes(fill='Semana'))
 ) ]

In [ ]:
Error_DF_Sum['Error %']=Error_DF_Sum.yhat/Error_DF_Sum.y-1
Error_DF_Sum.head()

In [ ]:
ggplot(Error_DF_Sum[Error_DF_Sum['Error %']<=2], aes(x='Semana',y='Error %'))+\
geom_col(aes(fill='Mes'))

In [ ]:
Error_DF_Sum.Error.describe()

#### group mensual 

In [ ]:
tmp2=Error_DF_Sum.copy()
tmp2.Error=tmp2.Error.astype('float64')#.abs()
tmp2.Mes=tmp2.Mes.astype('category')

print(tmp2.Error.mean())

dx=tmp2.groupby('Mes').sum().reset_index()#['Mes','Error']
dx

In [ ]:
ggplot(dx, aes(x=dx.Mes,y='Error'))+\
geom_col(aes(fill='Mes'))

In [ ]:
ggplot(dx, aes(x=dx.Mes,y='Error %'))+\
geom_col(aes(fill='Mes'))

In [ ]:
dx.Error.hist()